# Amazon-reviews predictions with Spark ML

In [1]:
import org.apache.spark.sql.SparkSession

val sc = SparkSession.builder.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_0199
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_0199)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
sc: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@53422aa9


## Read data and transform to RDD

In [89]:
val df = sc.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json").select("category", "reviewText")

df: org.apache.spark.sql.DataFrame = [category: string, reviewText: string]


In [90]:
val tokenizePattern = "[^a-zA-Z<>^|]+"

tokenizePattern: String = [^a-zA-Z<>^|]+


In [91]:
val stopWords = List(
  "a", "aa", "able", "about", "above", "absorbs", "accord", "according", "accordingly", "across",
  "actually", "after", "afterwards", "again", "against", "ain", "album", "all", "allow", "allows",
  "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an",
  "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway", "anyways", "anywhere",
  "apart", "app", "appear", "appreciate", "appropriate", "are", "aren", "around", "as", "aside", "ask",
  "asking", "associated", "at", "available", "away", "awfully", "b", "baby", "bb", "be", "became",
  "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "believe",
  "below", "beside", "besides", "best", "better", "between", "beyond", "bibs", "bike", "book", "books",
  "both", "brief", "bulbs", "but", "by", "c", "came", "camera", "can", "cannot", "cant", "car", "case",
  "cause", "causes", "cd", "certain", "certainly", "changes", "clearly", "co", "coffee", "com", "come",
  "comes", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains",
  "corresponding", "could", "couldn", "course", "currently", "d", "definitely", "described", "despite", "did",
  "didn", "different", "do", "does", "doesn", "dog", "dogs", "doing", "doll", "don", "done", "down", "downwards",
  "during", "e", "each", "edu", "eg", "eight", "either", "else", "elsewhere", "enough", "entirely", "especially",
  "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly",
  "example", "except", "f", "far", "few", "fifth", "film", "first", "five", "flavor", "followed", "following",
  "follows", "for", "former", "formerly", "forth", "four", "from", "fun", "further", "furthermore", "g", "game",
  "get", "gets", "getting", "given", "gives", "go", "goes", "going", "gone", "got", "gotten", "greetings", "grill",
  "guitar", "h", "had", "hadn", "hair", "happens", "hardly", "has", "hasn", "have", "haven", "having", "he", "hello",
  "help", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "hi", "him", "himself",
  "his", "hither", "hopefully", "how", "howbeit", "however", "i", "ie", "if", "ignored", "immediate", "in", "inasmuch", "inc",
  "indeed", "indicate", "indicated", "indicates", "ink", "inner", "insofar", "install", "instead", "into", "inward", "is", "isn",
  "it", "its", "itself", "j", "just", "k", "keep", "keeps", "kept", "kitchen", "knife", "know", "known", "knows", "l", "lamp",
  "laptop", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "life", "like", "liked", "likely",
  "little", "ll", "look", "looking", "looks", "ltd", "m", "mainly", "many", "may", "maybe", "me", "mean", "meanwhile", "merely",
  "might", "mon", "more", "moreover", "most", "mostly", "movie", "mower", "much", "must", "my", "myself", "n", "name", "namely",
  "nd", "near", "nearly", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "no", "nobody",
  "non", "none", "noone", "nor", "normally", "not", "nothing", "novel", "now", "nowhere", "o", "obviously", "of", "off", "often",
  "oh", "ok", "okay", "old", "on", "once", "one", "ones", "only", "onto", "or", "other", "others", "otherwise", "ought", "our", "ours",
  "ourselves", "out", "outside", "over", "overall", "own", "p", "particular", "particularly", "per", "perhaps", "phone", "placed", "please",
  "plus", "possible", "presumably", "printer", "probably", "product", "provides", "q", "que", "quite", "qv", "r", "rather", "rd", "re", "read",
  "really", "reasonably", "regarding", "regardless", "regards", "relatively", "respectively", "right", "s", "said", "same", "saw", "say",
  "saying", "says", "second", "secondly", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent",
  "serious", "seriously", "seven", "several", "shall", "shave", "she", "shoes", "should", "shouldn", "since", "six", "skin", "so", "some", "somebody",
  "somehow", "someone", "something", "sometime", "sometimes", "somewhat", "somewhere", "song", "songs", "soon", "sorry", "specified", "specify",
  "specifying", "still", "story", "strings", "stroller", "sub", "such", "sup", "sure", "t", "take", "taken", "taste", "tell", "tends", "th",
  "than", "thank", "thanks", "thanx", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter",
  "thereby", "therefore", "therein", "theres", "thereupon", "these", "they", "think", "third", "this", "thorough", "thoroughly", "those",
  "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "took", "toward", "towards", "toy", "tried", "tries",
  "truck", "truly", "try", "trying", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlikely", "until", "unto", "up", "upon",
  "us", "use", "used", "useful", "uses", "using", "usually", "v", "value", "various", "ve", "very", "via", "viz", "vs", "want", "wants", "was",
  "wasn", "way", "we", "wear", "welcome", "well", "went", "were", "weren", "what", "whatever", "when", "whence", "whenever", "where", "whereafter",
  "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
  "why", "will", "willing", "wish", "with", "within", "without", "won", "wonder", "would", "wouldn", "x", "y", "yes", "yet", "you", "your", "yours",
  "yourself", "yourselves", "z", "zero"
)


stopWords: List[String] = List(a, aa, able, about, above, absorbs, accord, according, accordingly, across, actually, after, afterwards, again, against, ain, album, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, app, appear, appreciate, appropriate, are, aren, around, as, aside, ask, asking, associated, at, available, away, awfully, b, baby, bb, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, bibs, bike, book, books, both, brief, bulbs, but, by, c, came, camera, can, cannot, cant, car, case, cause, causes, cd, certain, certainly, changes, clearly,...


### Helpers

In [100]:
import java.io.PrintWriter
import org.apache.spark.rdd.RDD

def writeRDDToFile(rdd: RDD[String], mergedDict: String, filePath: String) = {
    val writer = new PrintWriter(filePath)
    rdd.collect().foreach(line => writer.println(line))
    writer.println(mergedDict)
    writer.close()
}

import java.io.PrintWriter
import org.apache.spark.rdd.RDD
writeRDDToFile: (rdd: org.apache.spark.rdd.RDD[String], mergedDict: String, filePath: String)Unit


## Calculate Chi-Square

In [92]:
val counts = df.rdd.map(row => (row.getString(0), 1)).reduceByKey((a, b) => a + b)
val N = df.count()

counts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[146] at reduceByKey at <console>:46
N: Long = 78829


In [93]:
val countsAsMap = counts.collect.toMap

countsAsMap: scala.collection.immutable.Map[String,Int] = Map(Patio_Lawn_and_Garde -> 994, Movies_and_TV -> 4607, Electronic -> 7825, Office_Product -> 1243, Tools_and_Home_Improvement -> 1926, Kindle_Store -> 3205, Home_and_Kitche -> 4254, Digital_Music -> 836, Automotive -> 1374, Grocery_and_Gourmet_Food -> 1297, Baby -> 916, Book -> 22507, Clothing_Shoes_and_Jewelry -> 5749, Toys_and_Game -> 2253, Health_and_Personal_Care -> 2982, Sports_and_Outdoor -> 3269, Beauty -> 2023, CDs_and_Vinyl -> 3749, Musical_Instrument -> 500, Cell_Phones_and_Accessorie -> 3447, Apps_for_Android -> 2638, Pet_Supplie -> 1235)


In [94]:
%%time
val chiSquaredTermCategory = df.rdd.map(row => (row.getString(0), row.getString(1).toLowerCase().split(tokenizePattern).distinct))
                         .map(row => (row._1, row._2.filter(w => w.length > 1 && !stopWords.contains(w))))
                         .flatMap(row => row._2.map(term => (row._1, term)))
                         .map(row => (row, 1)).reduceByKey((a, b) => a + b)
                         .map(row => (row._1._2, (row._1._1, row._2))).groupByKey()
                         .map(row => (row._1, (row._2.map(x => x._2).sum, row._2)))
                         .flatMap(row => row._2._2.map(category => ((category._1), (row._1, category._2, row._2._1))))
                         .map(row => {    
                            val A = row._2._2
                            val B = row._2._3 - A
                            val C = countsAsMap(row._1) - A
                            val D = N - A - B - C
    
                            val chiSquared = (N * math.pow((A * D) - (B * C), 2)) / ((A + B) * (A + C) * (B + D) * (C + D))
                            (row._1, (row._2._1, chiSquared))
                        })
                        .groupByKey()
                        .map(row => (row._1, row._2.toList.sortBy(x => -x._2).take(75)))
                        .sortByKey()

Time: 18.433804988861084 seconds.



chiSquaredTermCategory: org.apache.spark.rdd.RDD[(String, List[(String, Double)])] = ShuffledRDD[168] at sortByKey at <console>:68


In [95]:
val mergedDict = chiSquaredTermCategory.flatMap(row => row._2.map(term => term._1)).distinct

mergedDict: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[172] at distinct at <console>:46


In [97]:
val result = chiSquaredTermCategory.map(row => {
    val key = row._1
    val values = row._2.map { case (str, num) => s"$str:$num" }.mkString(" ")
    s"<$key> $values"
})

result: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[173] at map at <console>:46


In [98]:
!hadoop fs -rm -r hdfs://captain01.os.hpc.tuwien.ac.at:9000/user/e12041500/output_rdd.txt

rm: `hdfs://captain01.os.hpc.tuwien.ac.at:9000/user/e12041500/output_rdd.txt': No such file or directory




In [99]:
%%time
result.coalesce(1).saveAsTextFile("output_rdd.txt")

Time: 0.8346476554870605 seconds.



In [101]:
%%time
writeRDDToFile(result, mergedDict.sortBy(x => x).reduce((a, b) => a + " " + b), "./output_rdd.txt")

Time: 0.41611814498901367 seconds.



## Datasets/DataFrames: Spark ML and Pipelines

In [37]:
import org.apache.spark.ml.feature.RegexTokenizer

val tokenizer = new RegexTokenizer()
  .setInputCol("reviewText")
  .setOutputCol("words")
  .setGaps(false)
  .setPattern(tokenizePattern)

import org.apache.spark.ml.feature.RegexTokenizer
tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_b13b5a1a2bbe, minTokenLength=1, gaps=false, pattern=[^a-zA-Z<>^|]+, toLowercase=true


In [39]:
import org.apache.spark.ml.feature.StopWordsRemover

val remover = new StopWordsRemover()
 .setInputCol("words")
 .setOutputCol("filtered")

<console>: 32: error: overloaded method constructor StopWordsRemover with alternatives:

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
// Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.

val hashingTF = new HashingTF()
  .setNumFeatures(1000)
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("features")
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.001)
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))

// Fit the pipeline to training documents.
val model = pipeline.fit(training)

// Now we can optionally save the fitted pipeline to disk
model.write.overwrite().save("/tmp/spark-logistic-regression-model")

// We can also save this unfit pipeline to disk
pipeline.write.overwrite().save("/tmp/unfit-lr-model")

// And load it back in during production
val sameModel = PipelineModel.load("/tmp/spark-logistic-regression-model")

// Prepare test documents, which are unlabeled (id, text) tuples.
val test = spark.createDataFrame(Seq(
  (4L, "spark i j k"),
  (5L, "l m n"),
  (6L, "spark hadoop spark"),
  (7L, "apache hadoop")
)).toDF("id", "text")

// Make predictions on test documents.
model.transform(test)
  .select("id", "text", "probability", "prediction")
  .collect()
  .foreach { case Row(id: Long, text: String, prob: Vector, prediction: Double) =>
    println(s"($id, $text) --> prob=$prob, prediction=$prediction")
  }